In [1]:
import csv
import os
import re
import multiprocessing
from urllib.request import urlretrieve

In [2]:
path = 'data/observations-182134.csv'

## Read entry data from CSV

In [3]:
licences = {}
scientific_names = {}
image_urls = []

with open(path) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        image_urls.append({
            'id': row['id'],
            'scientific_name': row['scientific_name'],
            'image_url': row['image_url']
        })
        if not row['license'] in licences:
            licences[row['license']] = 0
        licences[row['license']] += 1
        if not row['scientific_name'] in scientific_names:
            scientific_names[row['scientific_name']] = 0
        scientific_names[row['scientific_name']] += 1

### Licencing summary

In [4]:
licences

{'': 6443,
 'CC-BY-NC': 22707,
 'CC-BY': 22122,
 'CC-BY-SA': 1100,
 'CC-BY-NC-SA': 828,
 'CC-BY-NC-ND': 486,
 'CC0': 1116,
 'CC-BY-ND': 16}

### Scientific name summary

In [5]:
scientific_names

{'Entoloma virescens': 77,
 'Trametes coccinea': 1368,
 'Favolaschia calocera': 1112,
 'Auricularia cornea': 139,
 'Omphalotus nidiformis': 876,
 'Pulchrocladia retipora': 135,
 'Stereum versicolor': 756,
 'Microporus xanthopus': 248,
 'Lysurus mokusin': 152,
 'Clavulinopsis sulcata': 228,
 'Cladonia floerkeana': 61,
 'Podoscypha petalodes petalodes': 4,
 'Chlorophyllum brunneum': 136,
 'Coltricia australica': 170,
 'Nothojafnea cryptotricha': 3,
 'Lichenomphalia chromacea': 798,
 'Coprinellus micaceus': 61,
 'Cladonia confusa': 11,
 'Hygrocybe schistophila': 6,
 'Hypholoma brunneum': 129,
 'Aurantiporus pulcherrimus': 84,
 'Teloschistes chrysophthalmus': 192,
 'Laetiporus portentosus': 409,
 'Scutellinia scutellata': 8,
 'Rhizopus': 1,
 'Russula persanguinea': 385,
 'Aseroe rubra': 525,
 'Schizophyllum commune': 774,
 'Cantharellus concinnus': 253,
 'Amanita muscaria': 1307,
 'Sticta filix': 1,
 'Hericium coralloides': 135,
 'Macrolepiota clelandii': 453,
 'Leratiomyces ceres': 635,
 

In [6]:
len(scientific_names)

1156

In [7]:
len({k:v for k,v in scientific_names.items() if v >= 10})

481

In [8]:
len({k:v for k,v in scientific_names.items() if v >= 100})

138

In [9]:
sum([v for k,v in scientific_names.items() if v >= 10])

52986

We have 481 species with 10 or more images.
Let's 

Let's fetch only those species with 10 or more images.
That's 481 species and about 53k images.
Looks like we're seeing a long tail here - about half the species have only a few images.

## Filter entries to populated scientific names

In [10]:
popular_scientific_names = {k for k,v in scientific_names.items() if v >= 10}

In [11]:
populated_image_urls = [e for e in image_urls if e['scientific_name'] in popular_scientific_names]

In [12]:
len(populated_image_urls), len(image_urls)

(52986, 54818)

## Fetch images

In [13]:
# Directory scheme:
# data/scientific_name/inat_id.jpg

In [14]:
from urllib.parse import urlparse

In [15]:
def save_image(image_data):
    url = urlparse(image_data['image_url'])
    ext = re.search(r'\.(.*?)$', url.path).group(0).lower()
    filename = f'{image_data["id"]}{ext}'
    scientific_name = image_data['scientific_name'].lower().replace(' ', '_')
    outpath = os.path.join('data', scientific_name)
    outfile = os.path.join('data', scientific_name, filename)
    print(outfile)
    os.makedirs(outpath, exist_ok=True)
    urlretrieve(image_data['image_url'], outfile)

In [16]:
pool = multiprocessing.Pool(80)

data/lachnum_pteridophyllum/47958568.jpeg
data/tremella_fuciformis/47950679.jpg
data/hypholoma_brunneum/48073738.jpeg
data/gymnopilus_junonius/48090538.jpeg
data/roridomyces_austrororidus/48221485.jpeg
data/omphalotus_nidiformis/48107174.jpg
data/psilocybe_subaeruginosa/48216753.jpg
data/cortinarius_rotundisporus/48099048.jpeg
data/amanita_muscaria/48121954.jpeg
data/boletellus_obscurecoccineus/48082312.jpg
data/oudemansiella_gigaspora/48200718.jpg
data/bolbitius_titubans/47949286.jpeg
data/hymenoscyphus_berggrenii/47960840.jpeg
data/amanita_muscaria/48301243.jpeg
data/clathrus_archeri/48293211.jpg
data/rickenella_fibula/48237209.jpeg
data/lactarius_eucalypti/47954988.jpg
data/battarrea_phalloides/48210056.jpg
data/cortinarius_austrovenetus/48417018.jpeg
data/russula_lenkunya/48337978.jpeg
data/ramaria_lorithamnus/47980664.jpg
data/rhodofomitopsis_lilacinogilva/48420198.jpeg
data/heterotextus_peziziformis/48213831.jpeg
data/leucopaxillus_lilacinus/48311185.jpg
data/amanita_muscaria/484

data/ryvardenia_campyla/48966211.jpeg
data/ramariopsis_simplex/48424123.jpeg
data/gymnopilus_allantopus/48062617.jpeg
data/stereum_versicolor/48212445.jpeg
data/stereum_hirsutum/48624432.jpeg
data/urnula_campylospora/48398986.jpg
data/amanita_xanthocephala/48883353.jpg
data/cortinarius_archeri/48237234.jpeg
data/leotia_lubrica/48083964.jpg
data/gymnopilus_allantopus/48417024.jpeg
data/scytinotus_longinquus/48217245.jpeg
data/cruentomycena_viscidocruenta/48739518.jpg
data/leucopaxillus_eucalyptorum/48509795.jpg
data/lactarius_deliciosus/48633013.jpeg
data/lepista_nuda/48969591.jpg
data/russula_persanguinea/48320075.jpeg
data/lasiosphaeria_ovina/48762067.jpeg
data/artomyces_austropiperatus/48298814.jpeg
data/schizophyllum_commune/48425821.jpeg
data/clavaria_amoena/48630543.jpeg
data/lentinellus_pulvinulus/49002214.jpeg
data/ramariopsis_pulchella/48764503.jpeg
data/geastrum_triplex/48895998.jpeg
data/galerina_hypnorum/48524294.jpeg
data/gymnopilus_junonius/47955517.jpeg
data/stereum_illud

data/omphalotus_nidiformis/48243080.jpg
data/mycena_subgalericulata/49010202.jpeg
data/suillus_granulatus/48974661.jpg
data/phaeohelotium_baileyanum/48375967.jpeg
data/trametes_coccinea/48531886.jpeg
data/hypoxylon_howeanum/48215277.jpeg
data/geastrum_triplex/48630695.jpeg
data/russula_viridis/48489903.jpg
data/gliophorus_graminicolor/48427950.jpeg
data/lepista_sublilacina/48067830.jpg
data/amanita_xanthocephala/48326621.jpeg
data/podoscypha_petalodes/48625383.jpeg
data/mycena_interrupta/48633026.jpeg
data/pseudohydnum_gelatinosum/49003099.jpeg
data/oudemansiella_gigaspora/48605562.jpeg
data/mycena_epipterygia/48091427.jpg
data/mycena_mulawaestris/48323721.jpeg
data/sphaerobolus_stellatus/48315494.jpg
data/flammulina_velutipes/48086123.jpg
data/mycena_kuurkacea/48884675.jpeg
data/phaeotremella_fimbriata/48889078.jpeg
data/clavulinopsis_sulcata/48322074.jpeg
data/gymnopilus_allantopus/48075661.jpeg
data/omphalotus_nidiformis/47952886.jpg
data/ramaria_flaccida/48525988.jpeg
data/geastrum

data/mycena_clarkeana/48740106.jpg
data/collybia_eucalyptorum/48899530.jpeg
data/rickenella_fibula/48103368.jpg
data/stereum_versicolor/48633030.jpeg
data/aleuria_aurantia/48755306.jpeg
data/coltricia_australica/48110239.jpg
data/omphalotus_nidiformis/48307762.jpg
data/aurantiporus_pulcherrimus/48815220.jpg
data/trametes_coccinea/48205288.jpeg
data/volvopluteus_gloiocephalus/48277645.jpg
data/roridomyces_austrororidus/48398965.jpg
data/artomyces_austropiperatus/48630903.jpeg
data/cortinarius_austrovenetus/48743580.jpeg
data/pseudohydnum_gelatinosum/48632335.jpeg
data/cortinarius_kioloensis/48748950.jpg
data/kuehneromyces_brunneoalbescens/48186766.jpeg
data/clathrus_ruber/48407838.jpg
data/gymnopilus_allantopus/48492298.jpg
data/gymnopilus_ferruginosus/48421332.jpeg
data/inocybe_fibrillosibrunnea/48621259.jpeg
data/scleroderma_cepa/48298832.jpeg
data/deconica_horizontalis/48316217.jpeg
data/ramariopsis_crocea/48998904.jpeg
data/boletellus_emodensis/48629514.jpg
data/clavaria_amoena/4875

data/macrotyphula_juncea/48526172.jpeg
data/coltricia_australica/48748966.jpg
data/pholiota_squarrosipes/47954196.jpeg
data/heterotextus_peziziformis/49071106.jpeg
data/heterotextus_peziziformis/48621591.jpeg
data/phellodon_niger/48886703.jpeg
data/ramaria_lorithamnus/47949830.jpeg
data/phellodon_niger/48376548.jpeg
data/urnula_campylospora/47959966.jpeg
data/amanita_xanthocephala/48740416.jpg
data/leotia_lubrica/48525261.jpeg
data/armillaria_luteobubalina/48075967.jpeg
data/omphalotus_nidiformis/48093779.jpg
data/ascocoryne_sarcoides/48759755.jpeg
data/clavulinopsis_sulcata/48421338.jpeg
data/clavaria_amoena/48755940.jpeg
data/omphalotus_nidiformis/48219939.jpeg
data/volvopluteus_gloiocephalus/48530376.jpg
data/xanthoria_parietina/49004184.jpeg
data/cortinarius_archeri/48639331.jpg
data/lachnum_pteridophyllum/48213504.jpeg
data/pluteus_atromarginatus/49010908.jpeg
data/hymenotorrendiella_eucalypti/48324076.jpeg
data/ramariopsis_crocea/48318787.jpeg
data/pleurotus_purpureo-olivaceus/48

...

data/clavulinopsis_sulcata/50875314.jpeg
data/clavulina_rugosa/50976498.jpeg
data/macrolepiota_clelandii/50532933.jpeg
data/clavaria_zollingeri/51442869.jpeg
data/clavulinopsis_sulcata/51213842.jpeg
data/psilocybe_subaeruginosa/51556074.jpg
data/mycena_subgalericulata/51323752.jpg
data/ramaria_filicicola/50970617.jpeg
data/clavulinopsis_sulcata/50643721.jpeg
data/galerina_hypnorum/51411306.jpeg
data/laetiporus_portentosus/51205359.jpeg
data/mycena_interrupta/50248031.jpeg
data/nidula_niveotomentosa/50929467.jpeg
data/stereum_illudens/51074729.jpeg
data/leotia_lubrica/50254415.jpeg
data/russula_clelandii/51348736.jpg
data/chromocyphella_muscicola/51453633.jpeg
data/clavulinopsis_sulcata/50759587.jpeg
data/amanita_xanthocephala/51562893.jpg
data/lichenomphalia_chromacea/51675207.jpg
data/deconica_horizontalis/50765777.jpeg
data/coltriciella_dependens/50867320.jpeg
data/heterotextus_peziziformis/50933501.jpeg
data/pluteus_romellii/51071663.jpeg
data/conocybe_filaris/51680746.jpeg
data/rus

data/microporus_affinis/51068740.jpg
data/omphalotus_nidiformis/51545311.jpg
data/cruentomycena_viscidocruenta/51771025.jpg
data/cortinarius_rotundisporus/51976772.jpeg
data/amanita_xanthocephala/51541430.jpg
data/fomitiporia_robusta/51582073.jpg
data/amanita_muscaria/51851730.jpg
data/clavulinopsis_sulcata/51741144.jpeg
data/macrolepiota_clelandii/51325480.jpeg
data/sphaerobolus_stellatus/52003154.jpeg
data/banksiamyces_macrocarpus/51997515.jpeg
data/cortinarius_sinapicolor/51449413.jpeg
data/ascocoryne_sarcoides/51883102.jpeg
data/galerina_patagonica/51208982.jpeg
data/gymnopilus_junonius/52013792.jpg
data/ramaria_lorithamnus/51872557.jpeg
data/podoserpula_pusio/51347654.jpg
data/hypholoma_fasciculare_armeniacum/52100232.jpeg
data/byssomerulius_corium/51783419.jpeg
data/ramariopsis_crocea/51670931.jpeg
data/hygrocybe_astatogala/51559317.jpeg
data/russula_persanguinea/51952707.jpg
data/heterotextus_peziziformis/51459497.jpg
data/ramariopsis_crocea/51454097.jpeg
data/trametes_coccinea/

data/sanguinoderma_rude/52239611.jpeg
data/picipes_melanopus/52242940.jpeg
data/mycena_epipterygia/51989749.jpeg
data/tricholomopsis_rutilans/52004484.jpeg
data/tremella_fuciformis/52345990.jpeg
data/amanita_muscaria/51777189.jpg
data/geastrum_triplex/51219997.jpg
data/laetiporus_portentosus/51562361.jpg
data/trametes_coccinea/51766997.jpg
data/chlorophyllum_brunneum/51351633.jpg
data/clavulinopsis_sulcata/51780396.jpeg
data/mycena_subgalericulata/52122596.jpeg
data/gymnopilus_allantopus/52350083.jpeg
data/favolaschia_calocera/51889560.jpeg
data/hypholoma_fasciculare_armeniacum/51892763.jpeg
data/mycena_lazulina/51879613.jpeg
data/ramariopsis_kunzei/51454712.jpeg
data/heterotextus_peziziformis/51998158.jpg
data/hypholoma_fasciculare/51338880.jpeg
data/chlorociboria/52248463.jpg
data/lichenomphalia_chromacea/51671470.jpeg
data/clavulinopsis_sulcata/52227600.jpg
data/cortinarius_fibrillosus/51631715.jpeg
data/mycena_subgalericulata/51412010.jpeg
data/lactarius_eucalypti/51649796.jpg
data

data/mycena_cystidiosa/51767212.jpg
data/trametes_coccinea/51561187.jpg
data/amanita_xanthocephala/52001352.jpeg
data/clavulinopsis_sulcata/52340902.jpg
data/clavulina_vinaceocervina/51862990.jpeg
data/favolaschia_calocera/51774736.jpg
data/phaeoclavulina_abietina/52434741.jpeg
data/collybia_eucalyptorum/51449582.jpg
data/lichenomphalia_chromacea/51874157.jpeg
data/mycena_interrupta/52252727.jpg
data/chlorociboria/52222732.jpg
data/sanguinoderma_rude/52215248.jpg
data/clavulina_coralloides/52239857.jpeg
data/hypholoma_brunneum/51998436.jpg
data/phlebia_subceracea/51880678.jpeg
data/tremella_fuciformis/51767388.jpg
data/cortinarius_sinapicolor/52228893.jpg
data/aleuria_aurantia/51449818.jpeg
data/russula_lenkunya/52222881.jpg
data/sanguinoderma_rude/52215679.jpeg
data/entoloma_panniculus/52230838.jpg
data/amanita_xanthocephala/51767465.jpeg
data/hypholoma_brunneum/51880759.jpeg
data/hygrocybe_cheelii/51679216.jpeg
data/humidicutis_arcohastata/52415987.jpeg
data/stereum_versicolor/518833

data/trametes_coccinea/51995220.jpg
data/mycena_tuvara/52123298.jpeg


In [17]:
len(populated_image_urls)

52986

In [18]:
%time pool.map(save_image, populated_image_urls[:5000])

CPU times: user 7.37 s, sys: 6.07 s, total: 13.4 s
Wall time: 21min 24s


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [20]:
%time pool.map(save_image, populated_image_urls[5000:10000])

CPU times: user 7 s, sys: 5.28 s, total: 12.3 s
Wall time: 11min 6s


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [18]:
%time pool.map(save_image, populated_image_urls[10000:15000])

CPU times: user 6.07 s, sys: 4.53 s, total: 10.6 s
Wall time: 6min 2s


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [19]:
%time pool.map(save_image, populated_image_urls[15000:20000])

CPU times: user 4.53 s, sys: 3.26 s, total: 7.79 s
Wall time: 6min 26s


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [20]:
%time pool.map(save_image, populated_image_urls[20000:25000])

CPU times: user 4.56 s, sys: 3.48 s, total: 8.04 s
Wall time: 6min


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [ ]:
%time pool.map(save_image, populated_image_urls[25000:30000])

In [ ]:
%time pool.map(save_image, populated_image_urls[30000:35000])

In [ ]:
%time pool.map(save_image, populated_image_urls[35000:40000])

In [ ]:
%time pool.map(save_image, populated_image_urls[40000:45000])

In [ ]:
%time pool.map(save_image, populated_image_urls[45000:50000])

In [ ]:
%time pool.map(save_image, populated_image_urls[50000:])

### Time calculations

A sample run shows 12 seconds for 20 images

In [19]:
# Num hours for all images
50000/20*12/60/60

8.333333333333334

8 hours for all the images. Let's do them in batches of 5000.

In [20]:
# Approx time in minutes for 5000 images (pool 10)
5000/20*12/60

50.0

In [ ]:
# With pool of 20, 5000 takes 21 mins
# With pool of 40, 5000 takes 11 mins
# With pool of 80, 5000 takes 6 mins